In [0]:
-- One-time setup
CREATE SCHEMA IF NOT EXISTS stg;
CREATE SCHEMA IF NOT EXISTS dw;
CREATE SCHEMA IF NOT EXISTS mart;

In [0]:
-- MATCH
DROP TABLE stg.match_table;

CREATE TABLE IF NOT EXISTS stg.match_table (
  match_type_number     BIGINT   PRIMARY KEY,
  event_name            VARCHAR(256),
  match_stage           VARCHAR(64),
  event_date            DATE,
  event_year            INT,
  event_month           INT,
  event_day             INT,
  match_type            VARCHAR(64),
  season                VARCHAR(64),
  team_type             VARCHAR(32),
  overs                 INT,
  city                  VARCHAR(128),
  venue                 VARCHAR(256),
  gender                VARCHAR(16),
  first_team            VARCHAR(128),
  second_team           VARCHAR(128),
  match_result          VARCHAR(64),
  winner                VARCHAR(128),
  toss_winner           VARCHAR(128),
  toss_decision         VARCHAR(16),
  stg_file_name         VARCHAR(512),
  stg_file_row_number   INT,
  stg_file_hashkey      VARCHAR(256),
  stg_modified_ts       TIMESTAMP,
  ingest_date           DATE
)
DISTKEY(match_type_number)
SORTKEY(event_date, match_type_number);


-- PLAYER
DROP TABLE stg.player_table;

CREATE TABLE IF NOT EXISTS stg.player_table (
  match_type_number   BIGINT,
  country             VARCHAR(128),
  player_name         VARCHAR(256),
  stg_file_name       VARCHAR(512),
  stg_file_row_number INT,
  stg_file_hashkey    VARCHAR(256),
  stg_modified_ts     TIMESTAMP,
  ingest_date         DATE,
  PRIMARY KEY (match_type_number, country, player_name)
)
DISTKEY(match_type_number)
SORTKEY(match_type_number);

-- DELIVERY (with ball_in_over for uniqueness)
DROP TABLE stg.delivery_table;

CREATE TABLE IF NOT EXISTS stg.delivery_table (
  match_type_number   BIGINT,
  innings_number      INT,                -- NEW: distinguishes innings 1, 2, etc.
  country             VARCHAR(128),
  over                INT,
  ball_in_over        INT,
  bowler              VARCHAR(256),
  batter              VARCHAR(256),
  non_striker         VARCHAR(256),
  runs                INT,
  extras              INT,
  total               INT,
  player_out          VARCHAR(256),
  player_out_kind     VARCHAR(64),
  player_out_fielders VARCHAR(512),
  stg_file_name       VARCHAR(512),
  stg_file_row_number INT,
  stg_file_hashkey    VARCHAR(256),
  stg_modified_ts     TIMESTAMP,
  ingest_date         DATE,
  PRIMARY KEY (match_type_number, innings_number, over, ball_in_over)
)
DISTKEY(match_type_number)
SORTKEY(match_type_number, innings_number, over, ball_in_over);

In [0]:
COPY stg.match_table
FROM 's3://my-raw-odi-bucket/odis/csv/match_table/latest/'
IAM_ROLE 'arn:aws:iam::084828603834:role/service-role/AmazonRedshift-CommandsAccessRole-20250908T212717'
FORMAT AS PARQUET;

SELECT * FROM stg.match_table;

COPY stg.player_table
FROM 's3://my-raw-odi-bucket/odis/csv/player_table/latest/'
IAM_ROLE 'arn:aws:iam::084828603834:role/service-role/AmazonRedshift-CommandsAccessRole-20250908T212717'
FORMAT AS PARQUET;

COPY stg.delivery_table
FROM 's3://my-raw-odi-bucket/odis/csv/delivery_table/latest/'
IAM_ROLE 'arn:aws:iam::084828603834:role/service-role/AmazonRedshift-CommandsAccessRole-20250908T212717'
FORMAT AS PARQUET;

In [0]:
select * from stg.delivery_table where match_type_number = 1134 and innings_number = 2;